In [29]:
install.packages(c('randomForest', 'caret', 'doMC', 'plyr', 'reshape'))

Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/bin/macosx/el-capitan/contrib/3.4/randomForest_4.6-12.tgz': status was 'Couldn't resolve host name'”Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘randomForest’ failed”Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/bin/macosx/el-capitan/contrib/3.4/caret_6.0-77.tgz': status was 'Couldn't resolve host name'”Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘caret’ failed”Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/bin/macosx/el-capitan/contrib/3.4/doMC_1.3.4.tgz': status was 'Couldn't resolve host name'”Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘doMC’ failed”Warning message in downloa

In [2]:
genPrep <- function(seed){
  #Clean environment 
  rm(list=ls())
  gc()
  
  #include libraries
  library(caret)
  library(randomForest)
  library(doMC)
  library(plyr)
  library(reshape)
  
  #register DoMC cores
  registerDoMC(cores = 8)
}

splitData <- function(seed, data){
  # randomly splits data into training (0.8) and test (0.2) data
  
  set.seed(seed)
  training_index <- createDataPartition(data[[ncol(data)]], p = 0.8, list = F)
  training <- data[training_index,]
  test <- data[-training_index,]
  
  return(list(training=training,test=test))
}

In [3]:
trainModels <- function(dataset = NULL, outcomeVar = NULL, metric = "Accuracy", control = NULL){
  # trains all the caret models according to control and metric
  
  retList <- list()
  
  form <- as.formula(paste(c(outcomeVar, "~."), collapse=" "))
  
  #set.seed(seed)
  
  print("start train GLMNET")
  set.seed(seed)
  retList$glmnet <- caret::train(form, data=dataset, method="glmnet", metric=metric, preProc=c("center", "scale"), trControl=control)

  # KNN (KNN - k-nearest-neighbour)
  print("start train KNN")
  set.seed(seed)
  retList$knn <- caret::train(form, data=dataset, method="knn", metric=metric, preProc=c("center", "scale"), trControl=control)

  # nnet (ANN - artificial neural network)
  print("start train ANN")
  set.seed(seed)
  retList$ann <- caret::train(form, data=dataset, method="nnet", metric=metric, trControl=control)

  #C5.0 (DT - decision tree)
  print("start train DT")
  set.seed(seed)
  retList$dt <- caret::train(form, data=dataset, method="C5.0", metric=metric, trControl=control)

  # RF (RF - random forest)
  print("start train RF")
  set.seed(seed)
  retList$rf <- caret::train(form, data=dataset, method="rf", metric=metric, trControl=control)
  
  return(retList)
  
}

In [4]:
dataFile <- "iris_data_ml_omop_final.csv"


data <- read.csv(dataFile, header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
genPrep()

seed <- 7
splitData <- splitData(seed, data)
trainingData<-splitData$training
testData <- splitData$test
fileTrain <- "training.csv"
fileTest <- "test.csv"

write.table(trainingData, fileTrain, sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)
write.table(testData, fileTest, sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)

metric <- "Accuracy"
control <- trainControl(method="repeatedcv", number=10, repeats=10, sampling="down", savePredictions="all")
outcomeVar <- "human.body.structure"
trainedModels <- trainModels(trainingData, outcomeVar,metric,control)
modeltoSave <- 'myCreatedModels'
save(trainedModels,file = modeltoSave)

Warning message:
“package ‘caret’ was built under R version 3.4.1”Loading required package: lattice
Loading required package: ggplot2
Warning message in as.POSIXlt.POSIXct(Sys.time()):
“unknown timezone 'zone/tz/2017c.1.0/zoneinfo/Europe/Berlin'”randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Warning message:
“package ‘reshape’ was built under R version 3.4.1”
Attaching package: ‘reshape’

The following objects are masked from ‘package:plyr’:

    rename, round_any



[1] "start train GLMNET"


Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:reshape’:

    expand

Loaded glmnet 2.0-10



[1] "start train KNN"
[1] "start train ANN"
# weights:  48
initial  value 142.841981 
iter  10 value 125.517598
iter  20 value 57.703205
iter  30 value 32.302878
iter  40 value 15.723382
iter  50 value 14.780663
iter  60 value 14.239399
iter  70 value 12.953009
iter  80 value 12.403078
final  value 12.401622 
converged
[1] "start train DT"
[1] "start train RF"


In [11]:
createPredictModels <- function(dataset, modellist, packType = "caret"){
  ret <- list()
  
  for(model in modellist){
      modelName <- model$method
      ret[[modelName]] <- predict(model, dataset) 
    }
    
  return(ret)
}

evalModelListsAccsDf <- function(modellist, data, outcomeVar, packType = "caret", evalMetric = "Accuracy"){
  
  Model <- c();
  Accuracy <- c();
  cnt <- 1
  modelNames <- names(modellist)
  
  for (model in modellist){
      
      confMat <- confusionMatrix(model, data[,outcomeVar])
      Model[cnt] <- modelNames[cnt] 
      Accuracy[cnt] <- confMat[[c("overall",evalMetric)]]
      cnt <- cnt + 1
  }

    return(data.frame(Model, Accuracy))
  
  
}

predictFunc = function(){
     library(caret)
     modeltoLoad <- "myCreatedModels"
     load(modeltoLoad)
     training <- read.csv('training.csv', header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
     testData <- read.csv('test.csv', header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
     
     predictTrainCaretModels <- createPredictModels(training, trainedModels)
     predictTestCaretModels <- createPredictModels(testData, trainedModels)
     outcomeVar = 'human.body.structure'
     evalMetric <- "Accuracy"
     evalCaretModelsTraining <- evalModelListsAccsDf(predictTrainCaretModels, training, outcomeVar, evalMetric = evalMetric)
     evalCaretModelsTest <- evalModelListsAccsDf(predictTestCaretModels, testData, outcomeVar, evalMetric = evalMetric)
   
    #evalCaretModelsTraining
    evalCaretModelsTest

}

predictFunc()

Model,Accuracy
glmnet,1.0000000
knn,1.0000000
nnet,1.0000000
C5.0,0.9333333
rf,1.0000000


In [11]:
#ketos_predict
ketos_predict = function(){
    ketos_predict_data = ("tmpDummyUrl-will be set by system automatically")
    load("myCreatedModels")
    predict_data <- read.csv('test_input.csv', header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
    model = trainedModels$glmnet
    prediction = predict(model, predict_data)
    return (prediction)
}

as.character(ketos_predict())

[1] "bad body structure"